In [1]:
import sys
sys.path.append('../')
from SerbSynpretproceing import showGrid
from SerbainTagger import SrbTreeTagger
from srpskiwordnet import SrbWordNetReader
from srppolsets import PolaritySets, SrbSynset2GlossTransformer, syn2gloss
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from StemmerByNikola import stem_str
import numpy as np
from sklearn.neural_network import MLPClassifier
import pandas as pd

C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
C:\ProgramData\Anaconda3\envs\ML_extra\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [2]:
RES_DIR = "..\\resources\\"
def load_file_into_list(filename):
    with open(filename, mode="r", encoding="utf-16") as file:
        lines = [line.strip() for line in file]
    return lines

In [3]:
stop_words3 = load_file_into_list(RES_DIR + "stopwordsSRB tf.txt")
pos_df = pd.read_csv(RES_DIR + "pos.csv" )
neg_df = pd.read_csv(RES_DIR + "neg.csv" )
tt = SrbTreeTagger()


In [4]:
swordnet = SrbWordNetReader("F:\Serbian Corpora\wnsrp30","wnsrp30.xml")
Synset_Sentiment_set = PolaritySets(swordnet, 0)


Synset ENG30-07010821-n lacks literals 24612


In [5]:
Synset_Sentiment_set.addWSWN()


In [6]:
positive_seed_words = ["dobar",
                       "dobrota",
                       "lep",
                       "čudesno",
                       "dragocen",
                       "anđeoski",
                       "izobilje",
                       "izbavljenje",
                       "tešiti",
                       "ispravnost",
                       "oduševiti se",
                       "slast",
                       "uveseljavajući",
                       "napredovati",
                       "proslavljen",
                       "usrećiti",
                       "uspešnost"]

negative_seed_words = ["zao",
                       "antipatija",
                       "beda",
                       "bedan",
                       "bol",
                       "laž",
                       "lažno",
                       "korupcija",
                       "krivica",
                       "prezreti",
                       "tuga",
                       "nauditi",
                       "sebičnost",
                       "paćeništvo",
                       "ukloniti s ovog sveta",
                       "masakr",
                       "ratovanje"]
positive_seed_IDS = ["ENG30-01828736-v",
                     "ENG30-13987905-n",
                     "ENG30-01777210-v",
                     "ENG30-13987423-n",
                     "ENG30-01128193-v",
                     "ENG30-02355596-v",
                     "ENG30-00271946-v",
                    ]

negative_seed_IDS = ["ENG30-01510444-a",
                     "ENG30-01327301-v",
                     "ENG30-00735936-n",
                     "ENG30-00220956-a",
                     "ENG30-02463582-a",
                     "ENG30-01230387-a",
                     "ENG30-00193480-a",
                     "ENG30-00364881-a",
                     "ENG30-14213199-n",
                     "ENG30-01792567-v",
                     "ENG30-07427060-n",
                     "ENG30-14408086-n",
                     "ENG30-14365741-n",
                     "ENG30-02466111-a",
                     "ENG30-14204950-n",
                     "ENG30-10609960-n",
                     "ENG30-02319050-v",
                     "ENG30-02495038-v",
                     "ENG30-01153548-n",
                     "ENG30-00751944-n",
                    ]

Synset_Sentiment_set.addPOSall(positive_seed_words)
Synset_Sentiment_set.addNEGall(negative_seed_words)
Synset_Sentiment_set.addPOSIDall(positive_seed_IDS)
Synset_Sentiment_set.addNEGIDall(negative_seed_IDS)
Synset_Sentiment_set.addPOSIDall(pos_df["ID"])
Synset_Sentiment_set.addNEGIDall(neg_df["ID"])
rem_obj = ["ENG30-05528604-n",
"ENG30-00749767-n",
"ENG30-09593651-n",
"ENG30-13250542-n",
"ENG30-13132338-n",
"ENG30-05943066-n",
"ENG30-03123143-a",
"ENG30-10104209-n",
"ENG30-12586298-n",
"ENG30-01971094-n",
"ENG30-00759269-v",
"ENG30-00948206-n",
"ENG30-01039307-n",
"ENG30-02041877-v",
"ENG30-00023271-n",
"ENG30-13509196-n",
"ENG30-09450866-n",
"ENG30-03947798-n",
"ENG30-08589140-n"
          ]
Synset_Sentiment_set.removeSynIDs(rem_obj)


In [7]:
Synset_Sentiment_set.getDef()
Synset_Sentiment_set.updateDataFrame()

In [8]:
Synset_Sentiment_sets = list()
Synset_Sentiment_sets.append(Synset_Sentiment_set)

for _ in range(12):
    Synset_Sentiment_sets.append(
        Synset_Sentiment_sets[-1].next_itteration())

In [9]:
defi = list()

for s in Synset_Sentiment_sets:
    defi.append(s.getDef())

In [10]:
defi[12]['POS']

[{'ID': 'ENG30-00904290-a',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'izvanredan',
  'Definicija': 'koji zaslužuje najviše poštovanje i divljenje'},
 {'ID': 'ENG30-15245515-n',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'pravo vreme,trenutak,pravi čas',
  'Definicija': 'Pogodan trenutak.'},
 {'ID': 'ENG30-01818234-a',
  'POS': 0.0,
  'NEG': 0.875,
  'Lemme': 'negativan',
  'Definicija': 'Koga karakeriše ili koji pokazuje odbijanje, poricanje ili suprotstavljanje; koji nema pozitivna svojstva.'},
 {'ID': 'ENG30-05138208-n',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'vrednost',
  'Definicija': 'Kvalitet koji čini daa je nešto poželjno, vredno ili korisno.'},
 {'ID': 'ENG30-01275562-a',
  'POS': 0.875,
  'NEG': 0.0,
  'Lemme': 'važan',
  'Definicija': 'Od velikog značaja ili važnosti.'},
 {'ID': 'ENG30-04654337-n',
  'POS': 0.125,
  'NEG': 0.375,
  'Lemme': 'druželjubivost',
  'Definicija': 'prijateljska narav'},
 {'ID': 'ENG30-05154517-n',
  'POS': 0.625,
  'NEG': 0.0,
  'Lemme': 'preimućstvo,

In [11]:
defi[12]['NEG']

[{'ID': 'ENG30-01366157-a',
  'POS': 0.0,
  'NEG': 0.75,
  'Lemme': 'ožalošćen',
  'Definicija': 'koji ispoljava žalost'},
 {'ID': 'ENG30-04835028-n',
  'POS': 0.0,
  'NEG': 0.0,
  'Lemme': 'egocentričnost,egoizam,samoživost,sebičnost',
  'Definicija': 'Pokušaj da se za sebe dobije lično priznanje (posebno neprihvatljivim sredstvima).'},
 {'ID': 'ENG30-01780202-v',
  'POS': 0.0,
  'NEG': 0.625,
  'Lemme': 'strahovati,bojati se',
  'Definicija': 'biti uplašen, plašiti se'},
 {'ID': 'ENG30-01797051-v',
  'POS': 0.0,
  'NEG': 0.75,
  'Lemme': 'oplakati,tugovati,jadikovati,ožaliti',
  'Definicija': 'Osećati tugu.'},
 {'ID': 'ENG30-02451951-a',
  'POS': 0.0,
  'NEG': 0.75,
  'Lemme': 'nepopustljiv',
  'Definicija': 'Koji nije popustljiv, kojim se teško upravlja.'},
 {'ID': 'ENG30-01818234-a',
  'POS': 0.0,
  'NEG': 0.875,
  'Lemme': 'negativan',
  'Definicija': 'Koga karakeriše ili koji pokazuje odbijanje, poricanje ili suprotstavljanje; koji nema pozitivna svojstva.'},
 {'ID': 'ENG30-00118

In [12]:
defi[0]['NEG']

[{'ID': 'ENG30-01366157-a',
  'POS': 0.0,
  'NEG': 0.75,
  'Lemme': 'ožalošćen',
  'Definicija': 'koji ispoljava žalost'},
 {'ID': 'ENG30-04835028-n',
  'POS': 0.0,
  'NEG': 0.0,
  'Lemme': 'egocentričnost,egoizam,samoživost,sebičnost',
  'Definicija': 'Pokušaj da se za sebe dobije lično priznanje (posebno neprihvatljivim sredstvima).'},
 {'ID': 'ENG30-00969103-a',
  'POS': 0.0,
  'NEG': 0.625,
  'Lemme': 'gotski',
  'Definicija': 'koji je okarakterisan mračnim, mističnim i groteskom'},
 {'ID': 'ENG30-07386920-n',
  'POS': 0.0,
  'NEG': 0.625,
  'Lemme': 'mumljanje,mumlanje,mrmljanje',
  'Definicija': 'Tih neprekidan nerazumljiv zvuk; obično ga prati kretanje usana koje ne proivodi artikulisani govor.'},
 {'ID': 'ENG30-01797051-v',
  'POS': 0.0,
  'NEG': 0.75,
  'Lemme': 'oplakati,tugovati,jadikovati,ožaliti',
  'Definicija': 'Osećati tugu.'},
 {'ID': 'ENG30-02451951-a',
  'POS': 0.0,
  'NEG': 0.75,
  'Lemme': 'nepopustljiv',
  'Definicija': 'Koji nije popustljiv, kojim se teško upravl

In [13]:
defi[0]['POS']


[{'ID': 'ENG30-00904290-a',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'izvanredan',
  'Definicija': 'koji zaslužuje najviše poštovanje i divljenje'},
 {'ID': 'ENG30-15245515-n',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'pravo vreme,trenutak,pravi čas',
  'Definicija': 'Pogodan trenutak.'},
 {'ID': 'ENG30-01781478-a',
  'POS': 0.625,
  'NEG': 0.0,
  'Lemme': 'pobožan',
  'Definicija': 'Koji izražava ili propoveda obožavanje božanstva.'},
 {'ID': 'ENG30-00995775-a',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'povoljan,pogodan',
  'Definicija': 'koji ohrabruje, odobrava ili zadovoljava'},
 {'ID': 'ENG30-05138208-n',
  'POS': 0.75,
  'NEG': 0.0,
  'Lemme': 'vrednost',
  'Definicija': 'Kvalitet koji čini daa je nešto poželjno, vredno ili korisno.'},
 {'ID': 'ENG30-01275562-a',
  'POS': 0.875,
  'NEG': 0.0,
  'Lemme': 'važan',
  'Definicija': 'Od velikog značaja ili važnosti.'},
 {'ID': 'ENG30-05154517-n',
  'POS': 0.625,
  'NEG': 0.0,
  'Lemme': 'preimućstvo,plus',
  'Definicija': 'Korisna ili vr

In [14]:
pd.DataFrame(defi[0]['NEG']).to_csv("neg1.csv")

In [15]:
pp = [syn2gloss, tt.lemmarizer]
X, y = Synset_Sentiment_sets[0].getXY("POS", predprocess=pp)

In [16]:
X

0         koji zasluživati najviše poštovanje i divljenje 
1                                      pogodan trenutak . 
2        koji izražavati ili propovedati obožavanje bož...
3         koji ohrabrivati , odobravati ili zadovoljavati 
4        kvalitet koji činiti daa jesam nešto poželjan ...
                               ...                        
19470    dull-colored moth whose larvae Hava tufts of h...
19471    trava sa širok pljosnat lišće koji se gajiti u...
19472    knjiga štampan u velik tiraž koji se brzo prod...
19473    zelen mineral koji se sastojati od nikal-magne...
19474                           poslovan objekt za zabava 
Name: Sysnet, Length: 19841, dtype: object

In [17]:
y

0        1
1        1
2        1
3        1
4        1
        ..
19470    0
19471    0
19472    0
19473    0
19474    0
Name: POS, Length: 19841, dtype: int64

In [18]:
count = CountVectorizer(stop_words = stop_words3)

In [19]:
X_T = count.fit_transform(X)

In [20]:
X_T[y==1]

<147x19711 sparse matrix of type '<class 'numpy.int64'>'
	with 518 stored elements in Compressed Sparse Row format>

In [21]:
word_list = count.get_feature_names_out()

count_list = np.asarray(X_T.sum(axis=0))[0]

freq= (dict(zip(word_list, count_list)))

count_list = np.asarray(X_T[y==1].sum(axis=0))[0]

freq_1 = (dict(zip(word_list, count_list)))

count_list = np.asarray(X_T[y==0].sum(axis=0))[0]

freq_2 = (dict(zip(word_list, count_list)))

In [22]:
l=list(freq_1.items())
p = pd.DataFrame(l).sort_values(by=1, ascending = False)
p[p[1]>0]

,0,1
10426,osećanje,13
2469,dobar,11
10473,osobina,9
7045,kvalitet,9
8419,moralan,9
...,...,...
9075,negativan,1
17869,važno,1
11721,polni,1
13061,propadanje,1


In [23]:
l=list(freq_2.items())
p = pd.DataFrame(l).sort_values(by=1, ascending = False)
p[p[1]>0]

,0,1
14128,rod,897
6607,koristiti,876
1113,biljka,827
17888,velik,811
5648,jezik,779
...,...,...
8172,miksosporama,1
8171,miksobakterije,1
8250,misteriozan,1
8173,milava,1


In [24]:
l=list(freq.items())
pd.DataFrame(l)

,0,1
0,10,1
1,100l,1
2,1083c,1
3,10km,1
4,12,2
...,...,...
19706,еgipćanin,1
19707,еlamu,1
19708,еmpedokles,1
19709,еngleske,1


In [25]:
pp = [syn2gloss, tt.lemmarizer]
X, y = Synset_Sentiment_sets[0].getXY("NEG", predprocess=pp)
count = CountVectorizer(stop_words = stop_words3)
X_T = count.fit_transform(X)

word_list = count.get_feature_names_out()

count_list = np.asarray(X_T.sum(axis=0))[0]

freq= (dict(zip(word_list, count_list)))

count_list = np.asarray(X_T[y==1].sum(axis=0))[0]

freq_1 = (dict(zip(word_list, count_list)))

count_list = np.asarray(X_T[y==0].sum(axis=0))[0]

freq_2 = (dict(zip(word_list, count_list)))

In [26]:
l=list(freq_1.items())
p = pd.DataFrame(l).sort_values(by=1, ascending = False)
p[p[1]>0]

,0,1
10426,osećanje,17
16911,tuga,16
15642,stanje,16
1289,bol,13
5163,izazivati,11
...,...,...
7568,lož,1
5069,ispuštati,1
6208,klonuo,1
8664,nadržaj,1


In [27]:
l=list(freq_2.items())
p = pd.DataFrame(l).sort_values(by=1, ascending = False)
p[p[1]>0]

,0,1
14128,rod,897
6607,koristiti,881
1113,biljka,827
17888,velik,809
5648,jezik,779
...,...,...
8235,mirujućih,1
8240,misirski,1
8241,misisipi,1
8416,mor,1


In [28]:
count = CountVectorizer()
X_T = count.fit_transform(X)    

In [29]:
word_list = count.get_feature_names_out()
word_list

array(['10', '100l', '1083c', ..., 'еmpedokles', 'еngleske', 'еvropi'],
      dtype=object)